# RASER MRI Sim 5
> **Code Author**: Alon Greenbaum \
> **Last Edit**: 18 July 2025 \
> **Last Edited by**: Reagan McNeill Womack \

Code based on dynamics and theory outlined in [DOI: 10.1126/sciadv.abp8483](https://doi.org/10.1126/sciadv.abp8483)

*Google Gemini generative AI was used in the development of this notebook.*

## Requirements

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.ndimage import rotate
from pathlib import Path
import os
from datetime import datetime
import random
from skimage.transform import radon, iradon
from skimage.io import imsave

ModuleNotFoundError: No module named 'matplotlib'

## Results Directory
If directory does not exist, a new directory is made at location defined with `path`.

In [2]:
def makedirs(path):
  if not os.path.exists(path):
    os.makedirs(path)

## Sample Inversion Map
Create a sample 2D inversion map with a square shape for demonstration.

### Parameters
- `shape` (tuple of `int`, default: `(10,10)`: defines size of map, includes 2 variables (1 for each direction); represents the initial distribution of population inversion in a 2D space

### Returns
- A 2D NumPy array with dimensions specified by `shape`, a simplified population inversion map

In [3]:
def createSampleInversionMap(shape=(10,10)):
  squareSize = (shape[0] // 2, shape[1] // 2) # set size of square using parameter
  inversionMap = np.full(squareSize, 1)
  inversionMap = np.pad(inversionMap, ((shape[0] // 4, shape[0] // 4), (shape[1] // 4, shape[1] // 4)), mode="constant", constant_values=0)
  return inversionMap

array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 1, 1, 1, 0, 0],
       [0, 0, 1, 1, 1, 1, 1, 0, 0],
       [0, 0, 1, 1, 1, 1, 1, 0, 0],
       [0, 0, 1, 1, 1, 1, 1, 0, 0],
       [0, 0, 1, 1, 1, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]])

## Simulate RASER Dynamics
Core simulation function \
It models the time evolution of multiple interacting RASER (Radio-frequency Amplification by Stimulated Emission of Radiation) modes based on a set of coupled non-linear ordinary differential equations (ODEs), as described in the supplementary material (equations S5-S9). It takes an initial 1D population inversion profile and simulates how the population inversion, mode amplitudes, and phases change over time, ultimately calculating the total measurable output signal.

### Parameters
- \

### Returns: `dict`
A dictionary containing the results of the simulation.
- `time` (numpy.ndarray): a 1D array of time points (in seconds) at which the output signal was sampled
- `Nmodes` (int): the number of RASER modes/slices ($N$) simulated
- `initialInversion` (numpy.ndarray): the 1D array of initial population inversion values for each mode, as provided to the function
- `finalInversion` (numpy.ndarray): a 1D array of the population inversion values for each mode at the end of the simulation
- `finalAmplitude` (numpy.ndarray): a 1D array of the amplitude values ($A_\mu$) for each mode at the end of the simulation
- `outputSignal` (numpy.ndarray): a 1D array representing teh total simulated RASER signal ($Sig(t)$) (see Eq. S9) over the `simDuration`

## Test Functions
Comment/uncomment out functions to test they are working.

In [ ]:
# Sample Inversion map
createSampleInversionMap()